In [1]:
param_market = 'AU'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script></body></html>')[0] + ']}]}')
                
                # = = = = = = = = = = = = = = =
                
                market_id = json_data['o']['g']['marketplaceId'].strip()
                
                # = = = = = = = = = = = = = = =
                
                country = json_data['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = json_data['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = json_data['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = json_data['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US':
                    if market_id != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                if param_market == 'AU':
                    if market_id != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                elif param_market == 'UK':
                    if market_id != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                    
                # = = = = = = = = = = = = = = =
                
                specification = json_data['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if param_market in ['US', 'AU', 'UK']:
                    if 'VEHICLE_SELECTION' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                elif param_market == 'DE':
                    if 'MOTORS_TAB' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                        
                # = = = = = = = = = = = = = = =
                
                title = json_data['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in json_data['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = json_data['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'binModel' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in json_data['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = json_data['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if param_market in ['US', 'AU', 'UK']:
                            if 'available' in text['text']:
                                available = text['text'].strip()
                            elif 'sold' in text['text']:
                                sold = text['text'].strip()
                        elif param_market == 'DE':
                            if 'verfügbar' in text['text']:
                                available = text['text'].strip()
                            elif 'verkauft' in text['text']:
                                sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'action' in json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()
                else:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Item_Number': input_.loc[a, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Description_Url': description_url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                dict_name = {}
                if param_market in ['US', 'AU', 'UK']:
                    condition = 'Condition'
                elif param_market == 'DE':
                    condition = 'Artikelzustand'

                for name_value in specification:
                    name = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if name == condition:
                        df_temp[condition] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                    else:
                        name = name.lower()
                        if name in dict_name:
                            dict_name[name] += 1
                        else:
                            dict_name[name] = 1
                        df_temp[name if dict_name[name] == 1 else name + '-' + str(dict_name[name])] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Item_Number': input_.loc[a, 'Item_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./item_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1557

[ok] - 143441312796
[尝试次数：3] - [剩余数量：1545] - [当前时间：17:10:41]

[ok] - 143886900250
[尝试次数：6] - [剩余数量：1544] - [当前时间：17:10:41]

[ok] - 143866997741
[尝试次数：5] - [剩余数量：1543] - [当前时间：17:10:42]

[ok] - 142966761777
[尝试次数：1] - [剩余数量：1542] - [当前时间：17:10:42]

[ok] - 144925042379
[尝试次数：9] - [剩余数量：1541] - [当前时间：17:10:43]

[ok] - 142976868195
[尝试次数：7] - [剩余数量：1540] - [当前时间：17:10:44]

[ok] - 142966770490
[尝试次数：1] - [剩余数量：1539] - [当前时间：17:10:44]

[ok] - 142966769305
[尝试次数：2] - [剩余数量：1538] - [当前时间：17:10:44]

[ok] - 142911890466
[尝试次数：15] - [剩余数量：1537] - [当前时间：17:10:45]

[ok] - 144925093141
[尝试次数：14] - [剩余数量：1536] - [当前时间：17:10:45]

[ok] - 143372308662
[尝试次数：16] - [剩余数量：1535] - [当前时间：17:10:45]

[ok] - 145209757063
[尝试次数：1] - [剩余数量：1534] - [当前时间：17:10:46]

[ok] - 145209756409
[尝试次数：4] - [剩余数量：1533] - [当前时间：17:10:46]

[ok] - 145580510496
[尝试次数：1] - [剩余数量：1532] - [当前时间：17:10:46]

[ok] - 142919261710
[尝试次数：18] - [剩余数量：1531] - [当前时间：17:10:47]

[ok] - 145207867014
[尝试次数：10] - [剩余数量：1530] - [当前时间：17:1

[ok] - 143886986133
[尝试次数：1] - [剩余数量：1413] - [当前时间：17:11:49]

[ok] - 142973276485
[尝试次数：5] - [剩余数量：1412] - [当前时间：17:11:49]

[ok] - 142647342195
[尝试次数：3] - [剩余数量：1411] - [当前时间：17:11:49]

[ok] - 144800646120
[尝试次数：2] - [剩余数量：1410] - [当前时间：17:11:50]

[ok] - 142976868169
[尝试次数：2] - [剩余数量：1409] - [当前时间：17:11:50]

[ok] - 143887923323
[尝试次数：1] - [剩余数量：1408] - [当前时间：17:11:50]

[ok] - 144204935488
[尝试次数：2] - [剩余数量：1407] - [当前时间：17:11:50]

[ok] - 142269665124
[尝试次数：1] - [剩余数量：1406] - [当前时间：17:11:51]

[ok] - 143886988116
[尝试次数：6] - [剩余数量：1405] - [当前时间：17:11:52]

[ok] - 145207866880
[尝试次数：2] - [剩余数量：1404] - [当前时间：17:11:52]

[ok] - 145207866980
[尝试次数：1] - [剩余数量：1403] - [当前时间：17:11:53]

[ok] - 145207866954
[尝试次数：6] - [剩余数量：1402] - [当前时间：17:11:53]

[ok] - 145207866979
[尝试次数：3] - [剩余数量：1401] - [当前时间：17:11:53]

[ok] - 142922410134
[尝试次数：4] - [剩余数量：1400] - [当前时间：17:11:54]

[ok] - 145580510642
[尝试次数：2] - [剩余数量：1399] - [当前时间：17:11:54]

[ok] - 145580510644
[尝试次数：3] - [剩余数量：1398] - [当前时间：17:11:54]

[ok] - 1

[ok] - 144857740416
[尝试次数：6] - [剩余数量：1280] - [当前时间：17:12:43]

[ok] - 144905676034
[尝试次数：2] - [剩余数量：1279] - [当前时间：17:12:44]

[ok] - 144925042030
[尝试次数：2] - [剩余数量：1278] - [当前时间：17:12:44]

[ok] - 144905675856
[尝试次数：3] - [剩余数量：1277] - [当前时间：17:12:44]

[ok] - 142971938131
[尝试次数：1] - [剩余数量：1276] - [当前时间：17:12:45]

[ok] - 142971940176
[尝试次数：2] - [剩余数量：1275] - [当前时间：17:12:45]

[ok] - 142971936366
[尝试次数：3] - [剩余数量：1274] - [当前时间：17:12:46]

[ok] - 145207862844
[尝试次数：2] - [剩余数量：1273] - [当前时间：17:12:46]

[ok] - 143560692291
[尝试次数：1] - [剩余数量：1272] - [当前时间：17:12:47]

[ok] - 144925094136
[尝试次数：9] - [剩余数量：1271] - [当前时间：17:12:47]

[ok] - 142971942984
[尝试次数：4] - [剩余数量：1270] - [当前时间：17:12:48]

[ok] - 143560691609
[尝试次数：2] - [剩余数量：1269] - [当前时间：17:12:48]

[ok] - 142652218856
[尝试次数：23] - [剩余数量：1268] - [当前时间：17:12:50]

[ok] - 142652206105
[尝试次数：16] - [剩余数量：1267] - [当前时间：17:12:50]

[ok] - 145207867718
[尝试次数：3] - [剩余数量：1266] - [当前时间：17:12:51]

[ok] - 142968050493
[尝试次数：7] - [剩余数量：1265] - [当前时间：17:12:51]

[ok] -

[ok] - 143630220814
[尝试次数：3] - [剩余数量：1147] - [当前时间：17:13:48]

[ok] - 143380550272
[尝试次数：2] - [剩余数量：1146] - [当前时间：17:13:48]

[ok] - 143880602786
[尝试次数：4] - [剩余数量：1145] - [当前时间：17:13:49]

[ok] - 143877240309
[尝试次数：2] - [剩余数量：1144] - [当前时间：17:13:49]

[ok] - 143877209326
[尝试次数：1] - [剩余数量：1143] - [当前时间：17:13:49]

[ok] - 143872751283
[尝试次数：2] - [剩余数量：1142] - [当前时间：17:13:49]

[ok] - 145212838702
[尝试次数：1] - [剩余数量：1141] - [当前时间：17:13:50]

[ok] - 145212839017
[尝试次数：1] - [剩余数量：1140] - [当前时间：17:13:50]

[ok] - 142922410176
[尝试次数：3] - [剩余数量：1139] - [当前时间：17:13:50]

[ok] - 143109256005
[尝试次数：1] - [剩余数量：1138] - [当前时间：17:13:50]

[ok] - 142269675080
[尝试次数：1] - [剩余数量：1137] - [当前时间：17:13:51]

[ok] - 145212839277
[尝试次数：1] - [剩余数量：1136] - [当前时间：17:13:51]

[ok] - 145212839569
[尝试次数：3] - [剩余数量：1135] - [当前时间：17:13:51]

[ok] - 145580530868
[尝试次数：9] - [剩余数量：1134] - [当前时间：17:13:51]

[ok] - 145207869939
[尝试次数：1] - [剩余数量：1133] - [当前时间：17:13:51]

[ok] - 142265687991
[尝试次数：1] - [剩余数量：1132] - [当前时间：17:13:52]

[ok] - 1

[ok] - 142976868301
[尝试次数：3] - [剩余数量：1014] - [当前时间：17:14:44]

[ok] - 143887944811
[尝试次数：3] - [剩余数量：1013] - [当前时间：17:14:44]

[ok] - 144047718691
[尝试次数：2] - [剩余数量：1012] - [当前时间：17:14:45]

[ok] - 144047719845
[尝试次数：1] - [剩余数量：1011] - [当前时间：17:14:45]

[ok] - 144047719380
[尝试次数：3] - [剩余数量：1010] - [当前时间：17:14:45]

[ok] - 144053491307
[尝试次数：1] - [剩余数量：1009] - [当前时间：17:14:46]

[ok] - 144053491339
[尝试次数：1] - [剩余数量：1008] - [当前时间：17:14:46]

[ok] - 142966762006
[尝试次数：8] - [剩余数量：1007] - [当前时间：17:14:46]

[ok] - 144053491616
[尝试次数：2] - [剩余数量：1006] - [当前时间：17:14:47]

[ok] - 144053502161
[尝试次数：2] - [剩余数量：1005] - [当前时间：17:14:48]

[ok] - 144053502051
[尝试次数：1] - [剩余数量：1004] - [当前时间：17:14:48]

[ok] - 144053505660
[尝试次数：4] - [剩余数量：1003] - [当前时间：17:14:48]

[ok] - 144053505676
[尝试次数：1] - [剩余数量：1002] - [当前时间：17:14:49]

[ok] - 144053502089
[尝试次数：4] - [剩余数量：1001] - [当前时间：17:14:50]

[ok] - 145212642294
[尝试次数：1] - [剩余数量：1000] - [当前时间：17:14:51]

[ok] - 144053588419
[尝试次数：2] - [剩余数量：999] - [当前时间：17:14:51]

[ok] - 14

...

[ok] - 143564149662
[尝试次数：1] - [剩余数量：610] - [当前时间：17:18:09]

[ok] - 143870285562
[尝试次数：6] - [剩余数量：609] - [当前时间：17:18:10]

[ok] - 142647459391
[尝试次数：2] - [剩余数量：608] - [当前时间：17:18:10]

[ok] - 145207877484
[尝试次数：1] - [剩余数量：607] - [当前时间：17:18:11]

[ok] - 142911103268
[尝试次数：1] - [剩余数量：606] - [当前时间：17:18:11]

[ok] - 145207877590
[尝试次数：2] - [剩余数量：605] - [当前时间：17:18:12]

[ok] - 142273202702
[尝试次数：10] - [剩余数量：604] - [当前时间：17:18:13]

[ok] - 145207877302
[尝试次数：6] - [剩余数量：603] - [当前时间：17:18:13]

[ok] - 143524268382
[尝试次数：1] - [剩余数量：602] - [当前时间：17:18:13]

[ok] - 142933251563
[尝试次数：4] - [剩余数量：601] - [当前时间：17:18:14]

[ok] - 143872745527
[尝试次数：1] - [剩余数量：600] - [当前时间：17:18:15]

[ok] - 142973103521
[尝试次数：4] - [剩余数量：599] - [当前时间：17:18:16]

[ok] - 143872745347
[尝试次数：1] - [剩余数量：598] - [当前时间：17:18:16]

[ok] - 142973099301
[尝试次数：3] - [剩余数量：597] - [当前时间：17:18:17]

[ok] - 143380601368
[尝试次数：8] - [剩余数量：596] - [当前时间：17:18:17]

[ok] - 143872751336
[尝试次数：3] - [剩余数量：595] - [当前时间：17:18:18]

[ok] - 142973090631
[尝试

[ok] - 145580591616
[尝试次数：1] - [剩余数量：475] - [当前时间：17:19:26]

[ok] - 145212839586
[尝试次数：6] - [剩余数量：474] - [当前时间：17:19:26]

[ok] - 145212839599
[尝试次数：6] - [剩余数量：473] - [当前时间：17:19:26]

[ok] - 145212839306
[尝试次数：5] - [剩余数量：472] - [当前时间：17:19:27]

[ok] - 145217678256
[尝试次数：1] - [剩余数量：471] - [当前时间：17:19:28]

[ok] - 143866997531
[尝试次数：5] - [剩余数量：470] - [当前时间：17:19:29]

[ok] - 145221199569
[尝试次数：1] - [剩余数量：469] - [当前时间：17:19:29]

[ok] - 144697190735
[尝试次数：6] - [剩余数量：468] - [当前时间：17:19:29]

[ok] - 142265679193
[尝试次数：2] - [剩余数量：467] - [当前时间：17:19:30]

[ok] - 145580591638
[尝试次数：4] - [剩余数量：466] - [当前时间：17:19:30]

[ok] - 142265679195
[尝试次数：3] - [剩余数量：465] - [当前时间：17:19:31]

[ok] - 142787057659
[尝试次数：1] - [剩余数量：464] - [当前时间：17:19:31]

[ok] - 145212839047
[尝试次数：2] - [剩余数量：463] - [当前时间：17:19:31]

[ok] - 142787050887
[尝试次数：4] - [剩余数量：462] - [当前时间：17:19:32]

[ok] - 143558209041
[尝试次数：1] - [剩余数量：461] - [当前时间：17:19:32]

[ok] - 142533104024
[尝试次数：1] - [剩余数量：460] - [当前时间：17:19:32]

[ok] - 143681449962
[尝试次

[ok] - 143109300431
[尝试次数：2] - [剩余数量：339] - [当前时间：17:20:32]

[ok] - 141979119999
[尝试次数：1] - [剩余数量：338] - [当前时间：17:20:33]

[ok] - 145580586021
[尝试次数：9] - [剩余数量：337] - [当前时间：17:20:33]

[ok] - 143109300340
[尝试次数：1] - [剩余数量：336] - [当前时间：17:20:33]

[ok] - 145580586031
[尝试次数：11] - [剩余数量：335] - [当前时间：17:20:33]

[ok] - 143374955314
[尝试次数：3] - [剩余数量：334] - [当前时间：17:20:35]

[ok] - 143374958797
[尝试次数：3] - [剩余数量：333] - [当前时间：17:20:36]

[ok] - 142923398180
[尝试次数：19] - [剩余数量：332] - [当前时间：17:20:36]

[ok] - 143375102646
[尝试次数：5] - [剩余数量：331] - [当前时间：17:20:36]

[ok] - 143374963792
[尝试次数：5] - [剩余数量：330] - [当前时间：17:20:37]

[ok] - 143375102625
[尝试次数：3] - [剩余数量：329] - [当前时间：17:20:37]

[ok] - 142975577552
[尝试次数：19] - [剩余数量：328] - [当前时间：17:20:37]

[ok] - 143375103720
[尝试次数：3] - [剩余数量：327] - [当前时间：17:20:37]

[ok] - 143375103738
[尝试次数：2] - [剩余数量：326] - [当前时间：17:20:38]

[ok] - 142075255272
[尝试次数：9] - [剩余数量：325] - [当前时间：17:20:38]

[ok] - 143375104445
[尝试次数：2] - [剩余数量：324] - [当前时间：17:20:39]

[ok] - 142787057544
[

[ok] - 145207873168
[尝试次数：3] - [剩余数量：204] - [当前时间：17:21:28]

[ok] - 143797911364
[尝试次数：4] - [剩余数量：203] - [当前时间：17:21:28]

[ok] - 145207873687
[尝试次数：1] - [剩余数量：202] - [当前时间：17:21:29]

[ok] - 145207873691
[尝试次数：2] - [剩余数量：201] - [当前时间：17:21:29]

[ok] - 142930041543
[尝试次数：1] - [剩余数量：200] - [当前时间：17:21:30]

[ok] - 145395494988
[尝试次数：3] - [剩余数量：199] - [当前时间：17:21:30]

[ok] - 143626893197
[尝试次数：14] - [剩余数量：198] - [当前时间：17:21:31]

[ok] - 145395489539
[尝试次数：2] - [剩余数量：197] - [当前时间：17:21:31]

[ok] - 142973091041
[尝试次数：1] - [剩余数量：196] - [当前时间：17:21:31]

[ok] - 142930041606
[尝试次数：6] - [剩余数量：195] - [当前时间：17:21:32]

[ok] - 145596547318
[尝试次数：17] - [剩余数量：194] - [当前时间：17:21:32]

[ok] - 142933251386
[尝试次数：6] - [剩余数量：193] - [当前时间：17:21:32]

[ok] - 142973098966
[尝试次数：2] - [剩余数量：192] - [当前时间：17:21:33]

[ok] - 142973091109
[尝试次数：4] - [剩余数量：191] - [当前时间：17:21:33]

[ok] - 142933251474
[尝试次数：6] - [剩余数量：190] - [当前时间：17:21:34]

[ok] - 142973099018
[尝试次数：4] - [剩余数量：189] - [当前时间：17:21:34]

[ok] - 142973099070
[尝

[ok] - 142529706916
[尝试次数：4] - [剩余数量：69] - [当前时间：17:22:23]

[ok] - 143564179284
[尝试次数：11] - [剩余数量：68] - [当前时间：17:22:25]

[ok] - 142907375868
[尝试次数：4] - [剩余数量：67] - [当前时间：17:22:25]

[ok] - 142647421282
[尝试次数：3] - [剩余数量：66] - [当前时间：17:22:25]

[ok] - 142647375842
[尝试次数：4] - [剩余数量：65] - [当前时间：17:22:25]

[ok] - 142013099406
[尝试次数：1] - [剩余数量：64] - [当前时间：17:22:27]

[ok] - 142265542002
[尝试次数：2] - [剩余数量：63] - [当前时间：17:22:27]

[ok] - 142647375913
[尝试次数：6] - [剩余数量：62] - [当前时间：17:22:27]

[ok] - 145580585420
[尝试次数：13] - [剩余数量：61] - [当前时间：17:22:28]

[ok] - 143959352532
[尝试次数：13] - [剩余数量：60] - [当前时间：17:22:28]

[ok] - 145580516238
[尝试次数：4] - [剩余数量：59] - [当前时间：17:22:28]

[ok] - 142265678985
[尝试次数：4] - [剩余数量：58] - [当前时间：17:22:29]

[ok] - 142265678984
[尝试次数：2] - [剩余数量：57] - [当前时间：17:22:29]

[ok] - 142265679099
[尝试次数：4] - [剩余数量：56] - [当前时间：17:22:30]

[ok] - 143563301303
[尝试次数：6] - [剩余数量：55] - [当前时间：17:22:30]

[ok] - 143229011614
[尝试次数：2] - [剩余数量：54] - [当前时间：17:22:30]

[ok] - 145207873212
[尝试次数：2] - [剩余数量：